In [17]:
import open3d as o3d
import numpy as np
point_cloud = o3d.io.read_point_cloud("/content/vase_pc.ply")

# Maximum distance a point can be from the plane to be considered an inlier
distance_threshold = 0.01
# Number of points to sample
ransac_n = 3
# Number of iterations
num_iterations = 5000

plane_model, inliers = point_cloud.segment_plane(distance_threshold, ransac_n, num_iterations)
floor_points = point_cloud.select_by_index(inliers)

# Compute Covariance Matrix
centroid = np.mean(floor_points.points, axis=0)
centered_points = floor_points.points - centroid
covariance_matrix = np.cov(centered_points, rowvar=False)

# Compute Eigenvectors and Eigenvalues
eigenvalues, eigenvectors = np.linalg.eigh(covariance_matrix)

# Select Principal Axis
min_eigenvalue_index = np.argmin(eigenvalues)
principal_axis = eigenvectors[:, min_eigenvalue_index]

# Construct Rotation Matrix
v1 = principal_axis
v2 = np.array([0, 1, 0])  # Arbitrary orthogonal vector
v3 = np.cross(v1, v2)
rotation_matrix = np.column_stack((v1, v2, v3))

# Apply Rotation Matrix
rotated_points = np.dot(centered_points, rotation_matrix)

# Rotating full point cloud
rotated_pc = np.dot(point_cloud.points, rotation_matrix)

# Translating full point cloud
pc_centroid = np.mean(rotated_pc, axis=0)
pc_centered = rotated_pc - pc_centroid

rotated_point_cloud = o3d.geometry.PointCloud()
rotated_point_cloud.points = o3d.utility.Vector3dVector(pc_centered)

output_ply_file = "vase_pc_rotated.ply"
o3d.io.write_point_cloud(output_ply_file, rotated_point_cloud)

# Multiple saves and loads to save rotated point clouds
output_ply_file = "vase_pc_rotated.ply"
point_cloud = o3d.io.read_point_cloud(output_ply_file)	#rotated point cloud
pcd = o3d.geometry.PointCloud()
pcd.points = point_cloud.points

# Assigning random color and estimating normals for mesh creation
color = [1.0, 0.3, 0.5]
num_points = len(point_cloud.points)
colors = o3d.utility.Vector3dVector([color] * num_points)
pcd.colors = colors
point_cloud.estimate_normals()
pcd.normals = point_cloud.normals
print(pcd.points)
print(pcd.colors)
print(pcd.normals)

#Creating Mesh
mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, n_threads=1)
o3d.io.write_triangle_mesh("vase_mesh_rotated.obj", mesh)

std::vector<Eigen::Vector3d> with 157320 elements.
Use numpy.asarray() to access data.
std::vector<Eigen::Vector3d> with 157320 elements.
Use numpy.asarray() to access data.
std::vector<Eigen::Vector3d> with 157320 elements.
Use numpy.asarray() to access data.


True

In [ ]:
def unit_test():
    # Generate a random point cloud
    num_points = 1000
    points = np.random.rand(num_points, 3) * 10  # Random points in 3D space
    points[:, 1] = np.random.normal(loc=0, scale=0.1, size=num_points)  # Introduce noise in y-coordinates
    point_cloud = o3d.geometry.PointCloud()
    point_cloud.points = o3d.utility.Vector3dVector(points)

    # Apply random transformation (rotation and translation)
    rotation_matrix = o3d.geometry.get_rotation_matrix_from_xyz(np.random.uniform(-np.pi, np.pi, size=(3,)))
    translation_vector = np.random.uniform(-5, 5, size=(3,))
    point_cloud.transform(rotation_matrix)
    point_cloud.translate(translation_vector)

    # Align the entire point cloud to the YZ plane
    aligned_point_cloud = align_to_yz_plane(point_cloud)

    # Save the aligned point cloud to a PLY file for visualization
    o3d.io.write_point_cloud("aligned_point_cloud.ply", aligned_point_cloud)

# Run unit test
unit_test()